In [2]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep

In [3]:
#I like building
#I like building 
#I like building 

URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [4]:
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [5]:
parse(URL)

,Title,Location,Company,Salary,Synopsis
0,Clinical Data Analyst - CQE,None,NYU Langone Health,None,Work with NYULMC I...
1,Applied Machine Learning Scientist/Engineer - ...,None,Bloomberg,None,Information and da...
2,Lead Data Scientist,"New York, NY 10013 (Tribeca area)",Cherry Pick,"\n $80,000 - $120,000 a year",You will be the primary owner of t...
3,Data Scientist,"New York, NY 10018 (Clinton area)",Quartet,None,"As a Data Scientist at Quartet, yo..."
4,Data Scientist,"New York, NY",HBO,None,Work with data scientists and solu...
5,Data Scientist - NYC,"New York, NY",Daugherty Business Solutions,None,Proven data engineering or data vi...
6,Data Scientist,"New York, NY",Daily Harvest,None,Data Scientist at Daily Harvest. Experience di...
7,Data Warehouse Intern,"New York, NY",Newsela,None,"Work with Data Analysts, Data Engi..."
8,Bike Share Data Analyst,"Manhattan, NY",DEPARTMENT OF TRANSPORTATION,"\n $52,143 - $78,997 a year",Develop analyses of data based on ...
9,Junior Data Scientist,"New York, NY 10176 (Murray Hill area)",Dow Jones,None,"Junior Data Scientist, Membership ..."


In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                synopsis = None
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 100 == 0:  # Ram helped me build this counter to see how many. You can visibly see Ram's vernacular in the print statements.
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")
    sleep(2)

You have 100 results. 13 of these aren't rubbish.
You have 200 results. 21 of these aren't rubbish.
You have 300 results. 26 of these aren't rubbish.
You have 400 results. 28 of these aren't rubbish.
You have 500 results. 34 of these aren't rubbish.
You have 600 results. 47 of these aren't rubbish.
You have 700 results. 56 of these aren't rubbish.


In [22]:
df_more.head()

,Title,Location,Company,Salary,Synopsis
0,Data Scientist,None,eBay Inc.,None,We are looking for...
1,Jr. Computational Physicist/Scientist,None,"Nanohmics, Inc.","\n $45,000 - $75,000 a year",Experience with da...
2,Data Scientist,None,CyberCoders,None,If you are a Data ...
3,"Data Scientist, Austin","Austin, TX",Code Pilot,None,Design and implement data analysis...
4,Clinical Data Reporter,"Austin, TX 78731",Natera,None,Data collection and maintenance. Associate deg...


In [23]:
df_more.to_csv('uncleaned_indeed_file.csv', encoding='utf-8')

In [268]:
df_more = pd.read_csv('C:/Anaconda/webscraping/uncleaned_indeed_file.csv')

In [270]:
df_more.drop('Unnamed: 0', axis=1, inplace=True)

In [271]:
df_more = df_more[df_more.Salary != 'None'].drop_duplicates().dropna()

In [272]:
df_more = df_more[df_more.Salary.str.contains("hour") == False]
df_more = df_more[df_more.Salary.str.contains("month") == False]

In [273]:
def get_mean_salary(df, col):
    df[col].replace({'\$':''}, regex = True, inplace = True)
    df[col].replace(regex=True,inplace=True,to_replace=r'\D',value=r' ')
    df[col] = df[col].str.strip()
    df[col] = df_more[col].str.replace('   ',',')
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'min', 1:'max'}))
    df['min'] = df['min'].str.replace(' ','')
    df['min'] = df['min'].astype('float64')
    df['max'] = df['max'].str.replace(' ','')
    df['max'] = df['max'].astype('float64')
    df.drop(['Salary'], axis = 1, inplace = True)
    df['Salary'] = df[['min', 'max']].mean(axis=1)
    df.drop(['min','max'], axis=1, inplace = True)
    return df

In [274]:
df_more = get_mean_salary(df_more, 'Salary')

In [276]:
df_more.head()

,Title,Location,Company,Synopsis,Salary
18,Principal Data Scientist,"Austin, TX",Valkyrie Intelligence,Principal Scientist Position:. Pri...,142500.0
20,Senior Data Scientist,"Austin, TX",Valkyrie Intelligence,Senior Scientist Position:. Senior...,120000.0
21,Data Analyst - Public Education Data Analysis,"Austin, TX",Legislative Budget Board,The Data Analyst will coordinate a...,40000.0
32,Remote Job - Life Scientist Who Is Interested ...,"Austin, TX",Supreme Optimization,Analyze SEM/GA data to provide ins...,34000.0
35,Research Scientist (Neuroscience),"Austin, TX",Senseye,Manipulate and analyze large data ...,100000.0


In [278]:
def get_city_state(df,col):
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    df['State'] = [x[0:3] for x in df['State'] if x != None]
    df.drop(['Location'], axis = 1, inplace = True)
    return df

In [279]:
df_more = get_city_state(df_more, 'Location')

In [284]:
df_more.head()

,Title,Company,Synopsis,Salary,City,State
18,Principal Data Scientist,Valkyrie Intelligence,Principal Scientist Position:. Pri...,142500.0,Austin,TX
20,Senior Data Scientist,Valkyrie Intelligence,Senior Scientist Position:. Senior...,120000.0,Austin,TX
21,Data Analyst - Public Education Data Analysis,Legislative Budget Board,The Data Analyst will coordinate a...,40000.0,Austin,TX
32,Remote Job - Life Scientist Who Is Interested ...,Supreme Optimization,Analyze SEM/GA data to provide ins...,34000.0,Austin,TX
35,Research Scientist (Neuroscience),Senseye,Manipulate and analyze large data ...,100000.0,Austin,TX
